## import

In [1]:
import os
import torch
import argparse
import numpy as np
import torch.nn as nn
import torch.utils.data
import seaborn as sns
import imageio

from utils.dataset import load_mat_hsi, sample_gt, HSIDataset
from utils.utils import split_info_print, metrics, show_results
from utils.scheduler import load_scheduler
from models.get_model import get_model
from train import train, test
from utils.dynamic_tanh import convert_ln_to_dyt
import pandas as pd

/home1/jmt30269/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
print(os.getcwd())

/gpfs/home1/jmt30269/Group-Aware-Hierarchical-Transformer


## argument

In [3]:
parser = argparse.ArgumentParser(description="run patch-based HSI classification")
parser.add_argument("--model", type=str, default='proposed')
parser.add_argument("--dataset_name", type=str, default="gs")
parser.add_argument("--dataset_file_name", type=str, default="202407_downsampled_gongsan.h5")
parser.add_argument("--dataset_dir", type=str, default="/home1/jmt30269/DSNet/data/")
parser.add_argument("--device", type=str, default="0")
parser.add_argument("--patch_size", type=int, default=7)
parser.add_argument("--num_run", type=int, default=10) 
parser.add_argument("--epoch", type=int, default=200)    
parser.add_argument("--bs", type=int, default=128)  # bs = batch size  
parser.add_argument("--ratio", type=float, default=0.03)
parser.add_argument("--weights", type=str, default="/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_0")
parser.add_argument("--outputs", type=str, default="./results")


opts = parser.parse_args(args=[])

## main

In [8]:
device = torch.device("cuda:{}".format(opts.device))
print(device)
# print parameters
print("experiments will run on GPU device {}".format(opts.device))
print("model = {}".format(opts.model))    
print("dataset = {}".format(opts.dataset_name))
print("dataset folder = {}".format(opts.dataset_dir))
print("patch size = {}".format(opts.patch_size))
print("batch size = {}".format(opts.bs))
print("total epoch = {}".format(opts.epoch))
print("{} for training, {} for validation and {} testing".format(opts.ratio / 2, opts.ratio / 2, 1 - opts.ratio))

# load data
image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir,opts.dataset_file_name)

num_classes = len(labels)
num_bands = image.shape[-1]

# random seeds
seeds = [20250411, 20250402,20250403,20250404,20250405,20250406,20250407,20250408,20250409,20250410]

cuda:0
experiments will run on GPU device 0
model = proposed
dataset = gs
dataset folder = /home1/jmt30269/DSNet/data/
patch size = 7
batch size = 128
total epoch = 200
0.015 for training, 0.015 for validation and 0.97 testing


NameError: name 'load_mat_hsi' is not defined

## train

In [4]:
import time
# empty list to storing results
data_list= ['202307_downsampled_gongsan_x2.h5','202307_downsampled_gongsan_x4.h5','202307_downsampled_gongsan_x8.h5','202407_downsampled_gongsan_x2.h5','202407_downsampled_gongsan_x4.h5','202407_downsampled_gongsan_x8.h5',]
results = []
total_start_time = time.time() # 전체 실행 시작 시간
print("experiments will run on GPU device {}".format(opts.device))
print("model = {}".format(opts.model))    
for data_name in data_list:
    device = torch.device("cuda:{}".format(opts.device))
    opts.dataset_file_name=data_name
    
    # load data
    image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir,opts.dataset_file_name)
    
    num_classes = len(labels)
    num_bands = image.shape[-1] 
    # random seeds
    seeds = [20250411, 20250402,20250403,20250404,20250405,20250406,20250407,20250408,20250409,20250410]
    run=0
    run_start_time = time.time()
    np.random.seed(seeds[run])
    print("running an experiment with the {} data".format(opts.dataset_file_name))
    print("run {} / {}".format(run+1, opts.num_run))

    # get train_gt, val_gt and test_gt
    trainval_gt, test_gt = sample_gt(gt, opts.ratio, seeds[run])
    train_gt, val_gt = sample_gt(trainval_gt, 0.5, seeds[run])
    del trainval_gt

    train_set = HSIDataset(image, train_gt, patch_size=opts.patch_size, data_aug=True)
    val_set = HSIDataset(image, val_gt, patch_size=opts.patch_size, data_aug=False)

    train_loader = torch.utils.data.DataLoader(train_set, opts.bs, drop_last=False, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, opts.bs, drop_last=False, shuffle=False)

    # load model and loss
    model = get_model(opts.model, opts.dataset_name, opts.patch_size)
    if run == 0:
        split_info_print(train_gt, val_gt, test_gt, labels)
        # print("network information:")
        # with torch.no_grad():
        #     summary(model, torch.zeros((1, 1, num_bands, opts.patch_size, opts.patch_size)))
    model = model.to(device)
    optimizer, scheduler = load_scheduler(opts.model, model)

    criterion = nn.CrossEntropyLoss()

    # where to save checkpoint model
    model_dir = "./checkpoints/" + opts.model + '/' + opts.dataset_file_name + '/'+ str(opts.ratio)+'/' + str(opts.patch_size)+'_'+str(run)
    # model_dir = "./checkpoints/" + opts.model + '/' + opts.dataset_name + '/'+ str(opts.ratio)+'/' + str(opts.patch_size)+'_'+str(run)
    print(f'model save dir : {model_dir}')
    try:
        train(model, optimizer, criterion, train_loader, val_loader, opts.epoch, model_dir, device, scheduler)
    except KeyboardInterrupt:
        print('"ctrl+c" is pused, the training is over')

    run_end_time = time.time() # <<< 루프 반복 종료 시간 기록
    # test the model
    probabilities = test(model, model_dir, image, opts.patch_size, num_classes, device)
    
    prediction = np.argmax(probabilities, axis=-1)

    # computing metrics
    run_results = metrics(prediction, test_gt, n_classes=num_classes)  # only for test set
    results.append(run_results)
    show_results(run_results, label_values=labels)

    del model, train_set, train_loader, val_set, val_loader
    run_elapsed_time = run_end_time - run_start_time # <<< 현재 반복 실행 시간 계산
    print(f"--- Run {run+1} finished in {run_elapsed_time:.2f} seconds ---") # <<< 실행 시간 출력

total_end_time = time.time() # 전체 실행 종료 시간
total_elapsed_time = total_end_time - total_start_time
print(f"\nTotal execution time for {opts.num_run} runs: {total_elapsed_time:.2f} seconds")

if opts.num_run > 1:
    show_results(results, label_values=labels, agregated=True)

experiments will run on GPU device 0
model = proposed
running an experiment with the 202307_downsampled_gongsan_x2.h5 data
run 1 / 10
class train val test
AKK 898 898 58077
Background 14037 14038 907773
model save dir : ./checkpoints/proposed/202307_downsampled_gongsan_x2.h5/0.03/7_0


training the network:   0%|          | 0/200 [00:02<?, ?it/s]

train at epoch 1/200, loss=0.254311


training the network:   0%|          | 1/200 [00:03<12:34,  3.79s/it]

epoch = 1: best validation OA = 0.9521


...

train at epoch 190/200, loss=0.014083


training the network: 100%|██████████| 200/200 [00:42<00:00,  4.70it/s]
/gpfs/home1/jmt30269/Group-Aware-Hierarchical-Transformer/train.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experime

train at epoch 200/200, loss=0.011779


inference on the HSI: 100%|██████████| 1745/1745 [00:05<00:00, 311.65it/s]

Confusion matrix :
[[  442  2418]
 [  485 56807]]---
Accuracy : 95.17%
---
class acc :
	AKK: 15.45
	Background: 99.15
---
AA: 57.30%
Kappa: 21.52

--- Run 1 finished in 42.71 seconds ---

Total execution time for 10 runs: 1929.32 seconds
Agregated results :
Confusion matrix :
[[ 12247.83333333  10483.66666667]
 [  4536.66666667 395029.83333333]]---
Accuracy: 95.94 +- 0.65
---
class acc :
	AKK: 42.65 +- 18.67
	Background: 98.87 +- 0.22
---
AA: 70.76 +- 9.24
Kappa: 48.57 +- 16.84



## eval

In [6]:
def color_results(arr2d, palette):
    arr_3d = np.zeros((arr2d.shape[0], arr2d.shape[1], 3), dtype=np.uint8)
    for c, i in palette.items():
        m = arr2d == c
        arr_3d[m] = i
    return arr_3d

In [17]:

print("Missing keys:", missing_keys)
print("Unexpected keys:", unexpected_keys)

Alpha 파라미터 키들:
['block1.0.norm1.alpha', 'block1.0.norm2.alpha', 'norm1.alpha', 'block2.0.norm1.alpha', 'block2.0.norm2.alpha', 'norm2.alpha', 'block3.0.norm1.alpha', 'block3.0.norm2.alpha', 'norm3.alpha']
Missing keys: []
Unexpected keys: []


/tmp/ipykernel_237760/531262662.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(opts.weights+"_dyt", 'model_best.pth'))


In [25]:
palette = {0: (0, 0, 0)}
for k, color in enumerate(sns.color_palette("hls", num_classes + 1)):
    palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))

print('loading weights from %s' % opts.weights + '/model_best.pth')
model = get_model(opts.model, opts.dataset_name, opts.patch_size)
model = model.to(device)
# state_dict = torch.load(os.path.join(opts.weights+"_dyt", 'model_best.pth'))
# model.load_state_dict(state_dict)

model.eval()

# testing model: metric for the whole HSI, including train, val, and test
probabilities = test(model, opts.weights, image, opts.patch_size, num_classes, device=device)
prediction = np.argmax(probabilities, axis=-1)

run_results = metrics(prediction, gt, n_classes=num_classes)

prediction[gt < 0] = -1

# color results
colored_gt = color_results(gt+1, palette)
colored_pred = color_results(prediction+1, palette)

outfile = os.path.join(opts.outputs, opts.dataset_name,  opts.model)
os.makedirs(outfile, exist_ok=True)

# imageio.imsave(os.path.join(outfile, opts.dataset_name + '_gt.png'), colored_gt)  # eps or png
# imageio.imsave(os.path.join(outfile, opts.dataset_name+'_' + opts.model + '_out.png'), colored_pred)  # or png

prod = probabilities.reshape(-1,2)
gtt= gt.reshape(-1)
predd = prediction.reshape(-1)
gtt=gtt +1
predd=predd +1
df = pd.DataFrame({
    'gt': gtt,
    'prob_akk': prod[:, 0],  # 첫 번째 열
    'prob_back': prod[:, 1],  # 두 번째 열
    'pred': predd
})
# df.to_csv(os.path.join(outfile, opts.dataset_name+'_' + opts.model+"_result.csv"),index=False)

show_results(run_results, label_values=labels)
del model


loading weights from /home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_0/model_best.pth


inference on the HSI: 112092it [04:57, 376.91it/s]                            


Confusion matrix :
[[ 168667   71059]
 [  35567 3708441]]---
Accuracy : 97.32%
---
class acc :
	AKK: 70.36
	Background: 99.05
---
AA: 84.70%
Kappa: 74.58



## train size test

In [3]:
device = torch.device("cuda:{}".format(opts.device))
print("model = {}".format(opts.model))    
print("dataset = {}".format(opts.dataset_name))
print("dataset folder = {}".format(opts.dataset_dir))
print("patch size = {}".format(opts.patch_size))
print("batch size = {}".format(opts.bs))
print("total epoch = {}".format(opts.epoch))
opts.epoch=100
for i in np.arange(0.05, 0.2, 0.02):
    opts.ratio=i
    print("{} for training, {} for validation and {} testing".format(opts.ratio / 2, opts.ratio / 2, 1 - opts.ratio))

    # load data
    image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir,opts.dataset_file_name)
    
    num_classes = len(labels)
    num_bands = image.shape[-1]
    
    # random seeds
    seeds = [202201, 202202, 202203, 202204, 202205]
    
    # empty list to storing results
    results = []
    
    for run in range(opts.num_run):
        np.random.seed(seeds[run])
        print("running an experiment with the {} model".format(opts.model))
        print("run {} / {}".format(run+1, opts.num_run))
    
        # get train_gt, val_gt and test_gt
        trainval_gt, test_gt = sample_gt(gt, opts.ratio, seeds[run])
        train_gt, val_gt = sample_gt(trainval_gt, 0.5, seeds[run])
        del trainval_gt
    
        train_set = HSIDataset(image, train_gt, patch_size=opts.patch_size, data_aug=True)
        val_set = HSIDataset(image, val_gt, patch_size=opts.patch_size, data_aug=False)
    
        train_loader = torch.utils.data.DataLoader(train_set, opts.bs, drop_last=False, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, opts.bs, drop_last=False, shuffle=False)
    
        # load model and loss
        model = get_model(opts.model, opts.dataset_name, opts.patch_size)
    
        if run == 0:
            split_info_print(train_gt, val_gt, test_gt, labels)
            # print("network information:")
            # with torch.no_grad():
            #     summary(model, torch.zeros((1, 1, num_bands, opts.patch_size, opts.patch_size)))
        
        model = model.to(device)
        
        optimizer, scheduler = load_scheduler(opts.model, model)
    
        criterion = nn.CrossEntropyLoss()
    
        # where to save checkpoint model
        model_dir = "./checkpoints/" + opts.model + '/' + opts.dataset_name + '/'+ str(opts.ratio)+'/' + str(opts.patch_size)+'_'+str(run)
        print(f'model save dir : {model_dir}')
        try:
            train(model, optimizer, criterion, train_loader, val_loader, opts.epoch, model_dir, device, scheduler)
        except KeyboardInterrupt:
            print('"ctrl+c" is pused, the training is over')
    
        # test the model
        probabilities = test(model, model_dir, image, opts.patch_size, num_classes, device)
        
        prediction = np.argmax(probabilities, axis=-1)
    
        # computing metrics
        run_results = metrics(prediction, test_gt, n_classes=num_classes)  # only for test set
        results.append(run_results)
        show_results(run_results, label_values=labels)
    
        del model, train_set, train_loader, val_set, val_loader
    
    if opts.num_run > 1:
        show_results(results, label_values=labels, agregated=True)

model = proposed
dataset = gs
dataset folder = /home1/jmt30269/DSNet/data/
patch size = 7
batch size = 128
total epoch = 200
0.025 for training, 0.025 for validation and 0.95 testing
running an experiment with the proposed model
run 1 / 1
class train val test
AKK 5993 5993 227740
Background 93600 93600 3556808
model save dir : ./checkpoints/proposed/gs/0.05/7_0


training the network:   0%|          | 0/100 [00:13<?, ?it/s]

train at epoch 1/100, loss=0.120486


training the network:   1%|          | 1/100 [00:21<35:34, 21.56s/it]

epoch = 1: best validation OA = 0.9570


...

train at epoch 90/100, loss=0.028757


training the network:  99%|█████████▉| 99/100 [2:13:02<01:20, 80.13s/it]

train at epoch 100/100, loss=0.027103


training the network: 100%|██████████| 100/100 [2:13:32<00:00, 80.13s/it]
/gpfs/home1/jmt30269/Group-Aware-Hierarchical-Transformer/train.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experi

Confusion matrix :
[[ 169307   24871]
 [  26994 3005653]]---
Accuracy : 98.39%
---
class acc :
	AKK: 87.19
	Background: 99.11
---
AA: 93.15%
Kappa: 85.86



## test per size

In [32]:
import os
base_path='/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/'
list_dir=os.listdir(base_path)
list_dir.sort()
list_dir = [item for item in list_dir if '_dyt' in item]
print(list_dir)

['7_0_dyt', '7_1_dyt', '7_2_dyt', '7_3_dyt', '7_4_dyt', '7_5_dyt', '7_6_dyt', '7_7_dyt', '7_8_dyt', '7_9_dyt']


In [33]:
def color_results(arr2d, palette):
    arr_3d = np.zeros((arr2d.shape[0], arr2d.shape[1], 3), dtype=np.uint8)
    for c, i in palette.items():
        m = arr2d == c
        arr_3d[m] = i
    return arr_3d

In [36]:
for ratio in list_dir:
    opts.weights=base_path+ratio
    print(opts.weights)
    # load data
    image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir,opts.dataset_file_name)
    
    num_classes = len(labels)
    num_bands = image.shape[-1]

    # empty list to storing results
    results = []
    
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", num_classes + 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
    
    # load model and weights
    model = get_model(opts.model, opts.dataset_name, opts.patch_size)
    # print('loading weights from %s' % opts.weights + '/model_best.pth')
    model = model.to(device)
    model.load_state_dict(torch.load(os.path.join(opts.weights, 'model_best.pth')))
    model.eval()
    
    # testing model: metric for the whole HSI, including train, val, and test
    probabilities = test(model, opts.weights, image, opts.patch_size, num_classes, device=device)
    prediction = np.argmax(probabilities, axis=-1)
    
    run_results = metrics(prediction, gt, n_classes=num_classes)
    
    prediction[gt < 0] = -1
    
    # color results
    colored_gt = color_results(gt+1, palette)
    colored_pred = color_results(prediction+1, palette)
    
    outfile = os.path.join(opts.outputs, opts.dataset_name,  opts.model)
    os.makedirs(outfile, exist_ok=True)
    
    # imageio.imsave(os.path.join(outfile, opts.dataset_name + '_gt.png'), colored_gt)  # eps or png
    imageio.imsave(os.path.join(outfile, opts.dataset_name+'_' + opts.model+"_"+ratio+ '_out.png'), colored_pred)  # or png
    
    prod = probabilities.reshape(-1,2)
    gtt= gt.reshape(-1)
    predd = prediction.reshape(-1)
    gtt=gtt +1
    predd=predd +1
    df = pd.DataFrame({
        'gt': gtt,
        'prob_akk': prod[:, 0],  # 첫 번째 열
        'prob_back': prod[:, 1],  # 두 번째 열
        'pred': predd
    })
    df.to_csv(os.path.join(outfile, opts.dataset_name+'_' + opts.model+"_"+str(ratio)+"_result.csv"),index=False)
    
    show_results(run_results, label_values=labels)
    del model


/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_0_dyt


/tmp/ipykernel_237760/2730789655.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(opts.weights, 'model_best.pth'))
/gpfs/home1/jmt30

Confusion matrix :
[[ 180047   59679]
 [  41972 3702036]]---
Accuracy : 97.45%
---
class acc :
	AKK: 75.11
	Background: 98.88
---
AA: 86.99%
Kappa: 76.63

/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_1_dyt


/tmp/ipykernel_237760/2730789655.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(opts.weights, 'model_best.pth'))
/gpfs/home1/jmt30

Confusion matrix :
[[ 170925   68801]
 [  35852 3708156]]---
Accuracy : 97.37%
---
class acc :
	AKK: 71.30
	Background: 99.04
---
AA: 85.17%
Kappa: 75.18

/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_2_dyt


...

Confusion matrix :
[[ 173534   66192]
 [  38074 3705934]]---
Accuracy : 97.38%
---
class acc :
	AKK: 72.39
	Background: 98.98
---
AA: 85.69%
Kappa: 75.52

/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_8_dyt


/tmp/ipykernel_237760/2730789655.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(opts.weights, 'model_best.pth'))
/gpfs/home1/jmt30

Confusion matrix :
[[ 177499   62227]
 [  42520 3701488]]---
Accuracy : 97.37%
---
class acc :
	AKK: 74.04
	Background: 98.86
---
AA: 86.45%
Kappa: 75.82

/home1/jmt30269/Group-Aware-Hierarchical-Transformer/checkpoints/proposed/gs/0.03/7_9_dyt


/tmp/ipykernel_237760/2730789655.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(opts.weights, 'model_best.pth'))
/gpfs/home1/jmt30

Confusion matrix :
[[ 170332   69394]
 [  35555 3708453]]---
Accuracy : 97.37%
---
class acc :
	AKK: 71.05
	Background: 99.05
---
AA: 85.05%
Kappa: 75.06

